In [70]:
from pynq import Overlay
import time
ol = Overlay('matrix.bit')

In [3]:
#ol?

In [71]:
dma = ol.axi_dma_0

In [72]:
data_send = dma.sendchannel
data_recv = dma.recvchannel

In [73]:
import numpy as np
import random 

In [74]:
random.seed(2)
data_A = np.random.uniform(low=1, high=2, size=(32, 32)).astype(float)
data_B = np.random.uniform(low=1, high=2, size=(32, 32)).astype(float)

In [75]:
data_A, data_B

(array([[1.35540523, 1.9015954 , 1.72909453, ..., 1.77809696, 1.84822596,
         1.47975513],
        [1.93407165, 1.27019726, 1.88648091, ..., 1.5127058 , 1.20765421,
         1.38141031],
        [1.29582721, 1.31015558, 1.70945865, ..., 1.73287527, 1.21808493,
         1.33790179],
        ...,
        [1.90172866, 1.39270347, 1.89796209, ..., 1.08439472, 1.70552319,
         1.57903632],
        [1.62421922, 1.62320184, 1.62491543, ..., 1.63716749, 1.1012979 ,
         1.23947358],
        [1.04580102, 1.67661448, 1.43576529, ..., 1.61921871, 1.59340138,
         1.70233142]]),
 array([[1.60270689, 1.13669181, 1.75649831, ..., 1.47076429, 1.27498727,
         1.34346027],
        [1.68067822, 1.04254593, 1.67863652, ..., 1.80269848, 1.43131865,
         1.90968769],
        [1.67469241, 1.40637774, 1.47374611, ..., 1.2191692 , 1.32431405,
         1.30568718],
        ...,
        [1.8910264 , 1.54537781, 1.82955803, ..., 1.80268988, 1.51744763,
         1.42916529],
        [1.6

In [76]:
def multiply(matrix_a, matrix_b):
    result_matrix = np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            for k in range(8):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]
                
    return result_matrix

In [96]:
start_time = time.time()
multiply(data_A, data_B) #software
end_time = time.time()
print(end_time - start_time)

0.00411534309387207


In [78]:
from pynq import allocate

In [79]:
data_a_f = data_A.flatten()
data_b_f = data_B.flatten()

data_input = np.concatenate([data_a_f, data_b_f])

In [80]:
data_input

array([1.35540523, 1.9015954 , 1.72909453, ..., 1.79213653, 1.24404162,
       1.09059176])

In [82]:
input_buffer = allocate(32*32*2, np.float32)
output_buffer = allocate((32,32), np.float32)

In [83]:
np.copyto(input_buffer, data_input)

In [99]:
start_time = time.time()
data_send.transfer(input_buffer)
data_recv.transfer(output_buffer)
data_send.wait()
data_recv.wait()
end_time = time.time()
print(end_time - start_time)

0.002302885055541992


In [85]:
output_buffer

PynqBuffer([[70.61295 , 68.14172 , 77.79781 , ..., 71.09414 , 74.12363 ,
             72.29167 ],
            [68.80208 , 65.669945, 75.28384 , ..., 67.916565, 72.465225,
             69.13204 ],
            [66.40065 , 64.50014 , 73.34958 , ..., 65.51504 , 71.10686 ,
             67.7134  ],
            ...,
            [68.20529 , 65.4012  , 75.181496, ..., 68.846146, 72.76851 ,
             69.636   ],
            [66.51105 , 64.654625, 73.1568  , ..., 65.85971 , 70.50036 ,
             67.84411 ],
            [69.352516, 66.99948 , 75.74866 , ..., 69.33113 , 72.94032 ,
             69.70499 ]], dtype=float32)

In [86]:
diff = np.dot(data_A, data_B) - output_buffer

In [87]:
rms_error = np.sqrt(np.mean(diff**2))

In [88]:
diff, rms_error

(PynqBuffer([[-8.07020302e-06, -1.03570779e-05, -2.10341877e-06, ...,
               1.54530187e-06,  1.21737603e-05,  1.06357447e-06],
             [ 4.41643606e-06,  6.48833110e-06, -1.58808950e-05, ...,
              -2.41963582e-06,  1.66459112e-05,  3.01677784e-06],
             [-4.22144974e-06,  8.12530241e-06,  8.15668798e-07, ...,
               7.74689354e-07,  3.32025955e-06, -3.14817663e-06],
             ...,
             [ 4.01341946e-06,  4.92637068e-06, -1.86231004e-06, ...,
               1.61870078e-06,  1.59594947e-07,  1.61001134e-06],
             [ 1.16749550e-06, -4.28452414e-06, -7.02538890e-06, ...,
              -1.52816017e-06,  5.71798837e-06, -8.61427012e-07],
             [-1.21609204e-07,  1.87833939e-06, -4.98143349e-06, ...,
              -7.12786560e-06,  7.42221867e-06, -1.03505348e-05]]),
 PynqBuffer(6.18494001e-06))

In [94]:
%%time
np.dot(data_A, data_B)

CPU times: user 539 µs, sys: 0 ns, total: 539 µs
Wall time: 444 µs


array([[70.61294512, 68.14171328, 77.79781131, ..., 71.09414064,
        74.12363888, 72.29167282],
       [68.80208266, 65.66995125, 75.28382048, ..., 67.91656252,
        72.46524187, 69.13204495],
       [66.4006458 , 64.50014545, 73.34957967, ..., 65.51503831,
        71.10686062, 67.71339865],
       ...,
       [68.20529576, 65.40120427, 75.1814938 , ..., 68.84614725,
        72.76850907, 69.6360032 ],
       [66.51104853, 64.65462065, 73.15679229, ..., 65.85970917,
        70.5003643 , 67.84410772],
       [69.35251605, 66.99948308, 75.74865225, ..., 69.33112385,
        72.9403303 , 69.70497622]])

In [90]:
rms_error

PynqBuffer(6.18494001e-06)